# Preprocessing

## Featurizing

In [ ]:
import mdtraj as md
import numpy as np
import datetime
import tensorflow as tf
import nglview as nv


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import load_model

%cd /home/jovyan/ASMSA/mydev

import os, sys

repo_dir = os.getcwd()   
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

# 3. Fai l’import “pulito”
from utils import split_dataset, callbacks, plot_latent_space, process_trajectory
from vae import BetaVAE, BetaVAEMonitor, asmsa_beta_vae

nn_model = 'vae'
latent_dim = 2

In [ ]:
tr = "trpcage_ds_nH.xtc"
conf = "trpcage_npt400_nH.pdb"

traj = md.load_xtc(tr, top=conf)
backbone_atoms = traj.topology.select('backbone')
traj.superpose(traj, 0, atom_indices=backbone_atoms)

In [ ]:
view = nv.show_mdtraj(traj)

view.add_representation('line', selection='protein')
view

In [ ]:
ca_indices, n_ca, bb_indices, n_bb, features_normalized, scaler, coords = process_trajectory(tr, conf)
features_normalized.shape

## NN preprocessing

In [ ]:
# Uso:
ds_train, ds_val, ds_test, ds_all = split_dataset(features_normalized, train_size=70, val_size=15, batch_size=64, seed=42)

# VAE

In [ ]:
'''
Batch Norm, nel caso, va prima della layer activation)
'''

In [ ]:
log_dir = "logs/autoencoder/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

cb = callbacks(log_dir, latent_dim, monitor="val_loss")

learning_rate = 1e-4

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=learning_rate,
    weight_decay=1e-5, 
    beta_1=0.9,
    beta_2=0.999
)

# Creazione modello con beta migliorato
beta_vae, encoder, decoder = asmsa_beta_vae(
    n_features=features_normalized.shape[1], 
    latent_dim=latent_dim,
    beta=0.001 
)

# Compilazione
beta_vae.compile(optimizer=optimizer)

In [ ]:
beta_vae.fit(ds_train, epochs=500, validation_data=ds_val, callbacks=cb)

tensorboard --logdir logs/autoencoder --host localhost --port 6006

# Decode and visualize

In [ ]:
vae = load_model(
    'vae_2d.keras',
    custom_objects={
        'BetaVAE': BetaVAE,
        'BetaVAEMonitor': BetaVAEMonitor
    }
)
encoder = vae.get_layer('encoder')
decoder = vae.get_layer('decoder')


In [ ]:
for batch_x, _ in ds_test.take(1):
    sample_x = batch_x[30]  
    break
    
sample_x_batch = tf.expand_dims(sample_x, axis=0)
z_mean, z_log_var, test = encoder.predict(sample_x_batch)

test

In [ ]:
target = np.array([-2,0]).reshape(1, latent_dim)
emb, sample = plot_latent_space(latent_dim, encoder, ds_all, conf, tr, test, bb_indices, model=nn_model, exact=False)

In [ ]:
rms_ref = md.load_pdb(conf)
rms_ref_bb  = rms_ref.atom_slice(bb_indices)
rms_ref_ca  = rms_ref.atom_slice(ca_indices)
rms_tr = md.load_xtc(tr, top=rms_ref)
rmsd = md.rmsd(rms_tr, rms_ref)

In [ ]:
p_indices = traj.topology.select("protein")
n_p = len(p_indices)

In [ ]:
s = decoder.predict(sample)
s_orig = scaler.inverse_transform(s)

coords_flat = s_orig[0, :coords.shape[1]]                    
coords_p = coords_flat.reshape((n_p, 3))
mask_bb = np.isin(p_indices, bb_indices)
coords_bb = coords_p[mask_bb] 
coords_ca = coords_bb[1::4] 

new_traj = md.Trajectory(
    xyz=np.array([coords_bb]),     
    topology=rms_ref_bb.topology     
)


new_traj.save_pdb("vae_reconstructed.pdb")

In [ ]:
view = nv.show_file('vae_reconstructed.pdb')
view.clear_representations()
view.add_line() 
#view.add_cartoon()
view.center()
view

In [ ]:
mse = np.mean((s[0] - sample_x.numpy())**2)
print("MSE ricostruzione:", mse)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Serie originali e ricostruite
orig = sample_x.numpy()
recon = s[0]

# Calcolo errore per feature
error = orig - recon

plt.figure(figsize=(14, 6))

# 1️⃣ Confronto original vs ricostruito
plt.subplot(2, 1, 1)
plt.plot(orig, label="Originale", alpha=0.7)
plt.plot(recon, label="Ricostruito", alpha=0.7)
plt.title("Confronto Feature Originali vs Ricostruite")
plt.legend()

# 2️⃣ Errore per feature
plt.subplot(2, 1, 2)
plt.plot(error, label="Errore (Originale - Ricostruito)", color="red", alpha=0.7)
plt.axhline(0, color="black", linestyle="--", linewidth=0.8)
plt.title("Errore di Ricostruzione per Feature")
plt.legend()

plt.tight_layout()
plt.show()
